----------------
# <center>TP Spark 3 : Machine learning avec Spark</center>
-------------------

## Chargement du DataFrame

Charger le DataFrame obtenu à la fin du TP 2.

In [1]:
import org.apache.spark.sql.DataFrame

val df: DataFrame = spark
  .read
  .option("header", true) // utilise la première ligne du (des) fichier(s) comme header
  .option("inferSchema", "true") // pour inférer le type de chaque colonne (Int, String, etc.)
  .parquet("/home/p5hngk/Downloads/GitHub/INF_729---Introduction_au_framework_Hadoop/cours-spark-telecom-master/data/prepared_trainingset")

println("Training Dataframe")
df.show()

Intitializing Scala interpreter ...

Spark Web UI available at http://137.194.91.204:4040
SparkContext available as 'sc' (version = 2.4.4, master = local[*], app id = local-1573822096865)
SparkSession available as 'spark'


Training Dataframe
+--------------+--------------------+--------------------+------+--------------------+------------+--------+---------+-------------------+-------------------+-------------------+-------------+-----------+--------------------+
|    project_id|                name|                desc|  goal|            keywords|final_status|country2|currency2|          deadline2|        created_at2|       launched_at2|days_campaign|hours_prepa|                text|
+--------------+--------------------+--------------------+------+--------------------+------------+--------+---------+-------------------+-------------------+-------------------+-------------+-----------+--------------------+
| kkst471421639|american options ...|looking to create...|100000|american-options-...|           0|      US|      USD|2014-11-15 17:31:27|2014-10-10 21:23:58|2014-10-16 17:31:27|           30|    140.125|american options ...|
|kkst1098019088|iheadbones bone c...|wireless bluetoot...| 20000|iheadbones-b

import org.apache.spark.sql.DataFrame
df: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 12 more fields]


In [58]:
import org.apache.spark.ml.feature.{CountVectorizer, IDF, OneHotEncoderEstimator, RegexTokenizer, StringIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.StopWordsRemover

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, TrainValidationSplitModel}

import org.apache.spark.ml.feature.{CountVectorizer, IDF, OneHotEncoderEstimator, RegexTokenizer, StringIndexer}
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, TrainValidationSplitModel}


----------------------------
## Utilisation des données textuelles
Les textes ne sont pas utilisables tels quels par les algorithmes parce qu’ils ont besoin de données numériques, en particulier pour les calculs d’erreurs et d’optimisation. On veut donc convertir la colonne "text" en données numériques. Une façon très répandue de faire cela est d’appliquer l’algorithme [TF-IDF](https://spark.apache.org/docs/latest/ml-features.html#tf-idf).

### Stage 1 : récupérer les mots des textes
La première étape est de séparer les textes en mots (ou tokens) avec un tokenizer. Construire le premier stage du pipeline de la façon suivante :


In [4]:
val tokenizer = new RegexTokenizer()
  .setPattern("\\W+")
  .setGaps(true)
  .setInputCol("text")
  .setOutputCol("tokens")

tokenizer: org.apache.spark.ml.feature.RegexTokenizer = regexTok_f0d7cd885ec5


### Stage 2 : retirer les stops words

On veut retirer les [stop words](https://en.wikipedia.org/wiki/Stop_words) pour ne pas encombrer le modèle avec des mots qui ne véhiculent pas de sens. On va donc créer le 2ème stage avec la classe `StopWordsRemover`.

In [6]:
val stopWordsRemover = new StopWordsRemover()
  .setInputCol("tokens")
  .setOutputCol("filtered")

stopWordsRemover: org.apache.spark.ml.feature.StopWordsRemover = stopWords_e5c52a1d88c2


### Stage 3 : computer la partie TF
La partie TF de TF-IDF est faite avec la classe `CountVectorizer`. Lire la [doc](https://spark.apache.org/docs/latest/ml-features.html#tf-idf) pour plus d'info sur TF-IDF et son implémentation.

In [7]:
val countVectorizedModel = new CountVectorizer()
      .setInputCol("filtered")
      .setOutputCol("vectorized")

countVectorizedModel: org.apache.spark.ml.feature.CountVectorizer = cntVec_e5f43fbe87b3


### Stage 4 : computer la partie IDF
Implémentons la partie IDF avec en output une colonne ***tfidf***.

In [8]:
val idf = new IDF()
      .setInputCol("vectorized")
      .setOutputCol("tfidf")

idf: org.apache.spark.ml.feature.IDF = idf_5c82ac10ba3c


-------------------------
## Conversion des variables catégorielles en variables numériques

Les colonnes ***country2*** et ***currency2*** sont des variables catégorielles (qui ne prennent qu’un ensemble limité de valeurs, ces valeurs n'ayant, ici, aucune notion d'ordre entre elles), par opposition aux variables continues comme ***goal*** ou ***hours_prepa*** qui peuvent prendre n’importe quelle valeur réelle positive. Ici les catégories sont indiquées par une chaîne de charactères, e.g. "US" ou "EUR". On veut convertir ces classes en quantités numériques.

### Stage 5 : convertir ***country2*** en quantités numériques

Nous allons mettre les résultats dans une colonne ***country_indexed***.

In [10]:
val stringIndexer = new StringIndexer()
    .setInputCol("country2")
    .setOutputCol("country_indexed")
    .setHandleInvalid("skip")

stringIndexer: org.apache.spark.ml.feature.StringIndexer = strIdx_8b6ab1865213


### Stage 6 : convertir ***currency2*** en quantités numériques

Nous allons mettre les résultats dans une colonne ***currency_indexed***.

In [12]:
val stringIndexer2 = new StringIndexer()
      .setInputCol("currency2")
      .setOutputCol("currency_indexed")
      .setHandleInvalid("skip")

stringIndexer2: org.apache.spark.ml.feature.StringIndexer = strIdx_bc8d4a0514db


### Stage 7 et 8 : One-Hot encoder ces deux catégories 
Transformons ces deux catégories avec un "one-hot encoder" en créant les colonnes ***country_onehot*** et ***currency_onehot***. Une page [Quora](https://www.quora.com/What-is-one-hot-encoding-and-when-is-it-used-in-data-science) sur le one-hot encoding.

In [17]:
val oneHotEncoder = new OneHotEncoderEstimator()
      .setInputCols(Array("country_indexed", "currency_indexed"))
      .setOutputCols(Array("country_onehot", "currency_onehot"))

oneHotEncoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_9ad308b19850


---------------
## Mettre les données sous une forme utilisable par Spark.ML


La plupart des algorithmes de machine learning dans Spark requièrent que les colonnes utilisées en input du modèle (les features du modèle) soient regroupées dans une seule colonne qui contient des vecteurs. On veut donc passer de :

|Feature A|Feature B|Feature C|Label|
|:---:|:---:|:---:|:---:|
|0.5|1|3.5|0|
|0.6|1|1.2|1|

à

| Features | Label |
|:---:|:---:|
|(0.5, 1, 3.5)|0|
|(0.6, 1, 1.2)|1|

### Stage 9 : assembler tous les features en un unique vecteur

Assemblons les features ***tfidf***, ***days_campaign***, ***hours_prepa***, ***goal***, ***country_onehot***, et ***currency_onehot*** dans une seule colonne ***features***.

In [14]:
val vectorAssembler = new VectorAssembler()
      .setInputCols(Array("tfidf", "days_campaign", "hours_prepa", "goal", "country_onehot", "currency_onehot"))
      .setOutputCol("features")

println("OUTPUT FEATURES")

OUTPUT FEATURES


vectorAssembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_672416c38e7f


### Stage 10 : créer/instancier le modèle de classification

Le classifieur que nous utilisons est une [régression logistique](http://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.ml.classification.LogisticRegression) avec une régularisation dans la fonction de coût qui permet de pénaliser les features les moins fiables pour la classification.

On la définit de la façon suivante :


In [15]:
val lr = new LogisticRegression()
  .setElasticNetParam(0.0)
  .setFitIntercept(true)
  .setFeaturesCol("features")
  .setLabelCol("final_status")
  .setStandardization(true)
  .setPredictionCol("predictions")
  .setRawPredictionCol("raw_predictions")
  .setThresholds(Array(0.7, 0.3))
  .setTol(1.0e-6)
  .setMaxIter(20)

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_e2b134433876


-----------------------------
## Création du Pipeline

Créons maintenant le [pipeline](https://spark.apache.org/docs/latest/ml-pipeline.html#ml-pipelines) en assemblant les 10 stages définis précédemment, dans le bon ordre.

In [18]:
val stages10 = Array(tokenizer, stopWordsRemover, countVectorizedModel, idf, stringIndexer, stringIndexer2, oneHotEncoder, vectorAssembler, lr)
    val pipeline = new Pipeline().setStages(stages10)

stages10: Array[org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable}}}] = Array(regexTok_f0d7cd885ec5, stopWords_e5c52a1d88c2, cntVec_e5f43fbe87b3, idf_5c82ac10ba3c, strIdx_8b6ab1865213, strIdx_bc8d4a0514db, oneHotEncoder_9ad308b19850, vecAssembler_672416c38e7f, logreg_e2b134433876)
pipeline: org.apache.spark.ml.Pipeline = pipeline_c014b6d0b36d


## Entraînement, test, et sauvegarde du modèle

### Split des données en training et test sets

On veut séparer les données aléatoirement en un training set (90% des données) qui servira à l’entraînement du modèle et un test set (10% des données) qui servira à tester la qualité du modèle sur des données que le modèle n’a jamais vues lors de son entraînement. Cette phase est nécessaire pour avoir des résultats non-biaisés sur la pertinence du modèle obtenu.

Créons un DataFrame nommé **training** et un autre nommé **test** à partir du DataFrame chargé initialement de façon à le séparer en training et test sets dans les proportions 90%, 10% respectivement.


In [19]:
val Array(training, test) = df.randomSplit(Array(0.9, 0.1), seed = 1991)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [project_id: string, name: string ... 12 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [project_id: string, name: string ... 12 more fields]


### Entraînement du modèle

Entraînons notre modèle via le pipeline que nous avons créé puis sauvegardons-le.

In [20]:
val model = pipeline.fit(training)
println(s"Model 1 was fit using parameters: ${model.parent.extractParamMap}")

+--------------+--------------------+--------------------+-------+--------------------+------------+--------+---------+-------------------+-------------------+-------------------+-------------+-----------+--------------------+
|    project_id|                name|                desc|   goal|            keywords|final_status|country2|currency2|          deadline2|        created_at2|       launched_at2|days_campaign|hours_prepa|                text|
+--------------+--------------------+--------------------+-------+--------------------+------------+--------+---------+-------------------+-------------------+-------------------+-------------+-----------+--------------------+
|kkst1000004038|production elvis ...|a live stage prod...|  10000|production-elvis-...|           1|      US|      USD|2014-11-21 03:35:00|2014-09-26 02:03:58|2014-09-28 11:13:10|           54|     57.153|production elvis ...|
|kkst1000217634|haven's story: ti...|we proudly presen...|    150|havens-story-time...|     

model: org.apache.spark.ml.PipelineModel = pipeline_c014b6d0b36d


### Test du modèle

https://spark.apache.org/docs/latest/ml-pipeline.html

https://spark.apache.org/docs/latest/ml-tuning.html

* Appliquons le modèle aux données de test. Mettons les résultats dans le DataFrame `dfWithSimplePredictions`.

* Affichons
```scala
dfWithSimplePredictions.groupBy("final_status", "predictions").count.show()
```



In [21]:
val dfWithSimplePredictions = model.transform(test)

dfWithSimplePredictions.groupBy("final_status", "predictions").count.show()

+------------+-----------+-----+
|final_status|predictions|count|
+------------+-----------+-----+
|           1|        0.0| 1810|
|           0|        0.0| 5016|
|           1|        1.0| 1600|
|           0|        1.0| 2376|
+------------+-----------+-----+



dfWithSimplePredictions: org.apache.spark.sql.DataFrame = [project_id: string, name: string ... 24 more fields]


* Affichons le [*f1-score*](https://en.wikipedia.org/wiki/F1_score) du modèle sur les données de test (cette métrique s'obtient via [MulticlassClassificationEvaluator](https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator)).

In [23]:
val evaluator = new MulticlassClassificationEvaluator()
    .setMetricName("f1")
    .setLabelCol("final_status")
    .setPredictionCol("predictions")

evaluator: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_d6f6bd8301b5


In [28]:
val f1score = evaluator.evaluate(dfWithSimplePredictions)

println("Le f1-score est de " + f1score)

Le f1-score est de 0.6196139660457709


f1score: Double = 0.6196139660457709


----------------------
## Réglage des hyper-paramètres (a.k.a. tuning) du modèle

La façon de procéder présentée plus haut permet rapidement d'entraîner un modèle et d'avoir une mesure de sa performance. Mais que se passe-t-il si l'ont souhaite utiliser 300 itérations au maximum plutôt que 50 (i.e. la ligne `.setMaxIter(50)`) ? Si l'on souhaite modifier le paramètre de régularisation du modèle ? Si l'on souhaite modifier le paramètre *minDF* de la classe `CountVectorizer` (qui permet de ne prendre que les mots apparaissant dans au moins minDF documents) ? Il faudrait à chaque fois modifier le(s) paramètre(s) à la main, ré-entraîner le modèle, re-calculer la performance du modèle obtenu sur l'ensemble de test, puis finalement choisir le meilleur modèle (i.e. celui avec la meilleure performance sur les données de test) parmi tous ces modèles entraînés. C'est ce qu'on appelle le réglage des hyper-paramètres ou encore tuning du modèle. Et c'est fastidieux.

La plupart des algorithmes de machine learning possèdent des hyper-paramètres, par exemple le nombre de couches et de neurones dans un réseau de neurones, le nombre d’arbres et leur profondeur maximale dans les random forests, etc. Qui plus est, comme mentionné précédemment avec le paramètre *minDF* de la classe `CountVectorizer`, on peut également se retrouver avec des hyper-paramètres au niveau des stages de préprocessing. L'objectif est donc de trouver la meilleure combinaison possible de tous ces hyper-paramètres.

### Grid search

Une des techniques pour régler automatiquement les hyper-paramètres est la *grid search* qui consiste à :
- créer une grille de valeurs à tester pour les hyper-paramètres
- en chaque point de la grille
    - séparer le training set en un ensemble de training (70%) et un ensemble de validation (30%)
    - entraîner un modèle sur le training set
    - calculer l’erreur du modèle sur le validation set
- sélectionner le point de la grille (<=> garder les valeurs d’hyper-paramètres de ce point) où l’erreur de validation est la plus faible i.e. là où le modèle a le mieux appris

Pour la régularisation de notre régression logistique on veut tester les valeurs de 10e-8 à 10e-2 par pas de 2.0 en échelle logarithmique (on veut tester les valeurs 10e-8, 10e-6, 10e-4 et 10e-2).
Pour le paramètre minDF de CountVectorizer on veut tester les valeurs de 55 à 95 par pas de 20. 
En chaque point de la grille on veut utiliser 70% des données pour l’entraînement et 30% pour la validation.
On veut utiliser le *f1-score* pour comparer les différents modèles en chaque point de la grille.

Préparons la grid-search pour satisfaire les conditions explicitées ci-dessus puis lançons la grid-search sur le dataset "training" préparé précédemment.

In [30]:
val paramGrid = new ParamGridBuilder()
    .addGrid(lr.regParam, Array(10e-8, 10e-6, 10e-4, 10e-2))
    .addGrid(countVectorizedModel.minDF, Array(55.0, 75.0, 95.0))
    .build()

paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	cntVec_e5f43fbe87b3-minDF: 55.0,
	logreg_e2b134433876-regParam: 1.0E-7
}, {
	cntVec_e5f43fbe87b3-minDF: 75.0,
	logreg_e2b134433876-regParam: 1.0E-7
}, {
	cntVec_e5f43fbe87b3-minDF: 95.0,
	logreg_e2b134433876-regParam: 1.0E-7
}, {
	cntVec_e5f43fbe87b3-minDF: 55.0,
	logreg_e2b134433876-regParam: 1.0E-5
}, {
	cntVec_e5f43fbe87b3-minDF: 75.0,
	logreg_e2b134433876-regParam: 1.0E-5
}, {
	cntVec_e5f43fbe87b3-minDF: 95.0,
	logreg_e2b134433876-regParam: 1.0E-5
}, {
	cntVec_e5f43fbe87b3-minDF: 55.0,
	logreg_e2b134433876-regParam: 0.001
}, {
	cntVec_e5f43fbe87b3-minDF: 75.0,
	logreg_e2b134433876-regParam: 0.001
}, {
	cntVec_e5f43fbe87b3-minDF: 95.0,
	logreg_e2b134433876-regParam: 0.001
}, {
	cntVec_e5f43fbe87b3-minDF: 55.0,
	logreg_e2...

In [31]:
//  TrainValidationSplit requiert un estimateur, un set d'estimateur ParamMaps, et un Evaluator.
val trainValidationSplit = new TrainValidationSplit()
    .setEstimator(pipeline)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(paramGrid)
    .setTrainRatio(0.7)

trainValidationSplit: org.apache.spark.ml.tuning.TrainValidationSplit = tvs_fed7ded53f02


In [32]:
// Entrainement du modèle avec l'échantillon training
println("Entrainement du modèle avec l'échantillon training")
val validationModel = trainValidationSplit.fit(training)

Entrainement du modèle avec l'échantillon training


validationModel: org.apache.spark.ml.tuning.TrainValidationSplitModel = tvs_fed7ded53f02


### Test du modèle

On a vu que pour évaluer de façon non biaisée la pertinence du modèle obtenu, il fallait le tester sur des données qu'il n'avait jamais vues pendant son entraînement. Ça vaut également pour les données utilisées pour sélectionner le meilleur modèle de la grid search (training et validation)! C’est pour cela que nous avons construit le dataset de test que nous avons laissé de côté jusque là.

* Appliquons le meilleur modèle trouvé avec la grid-search aux données de test. Mettons les résultats dans le DataFrame `dfWithPredictions`. Affichons le f1-score du modèle sur les données de test.

* Affichons
```scala
dfWithPredictions.groupBy("final_status", "predictions").count.show()
```


* Sauvegardons le modèle entraîné pour pouvoir le réutiliser plus tard.


In [34]:
val dfWithPredictions = validationModel.transform(test).select("features","final_status","predictions")

dfWithPredictions.groupBy("final_status", "predictions").count.show()

+------------+-----------+-----+
|final_status|predictions|count|
+------------+-----------+-----+
|           1|        0.0| 1049|
|           0|        0.0| 4493|
|           1|        1.0| 2361|
|           0|        1.0| 2899|
+------------+-----------+-----+



dfWithPredictions: org.apache.spark.sql.DataFrame = [features: vector, final_status: int ... 1 more field]


In [80]:
val score = evaluator.evaluate(dfWithPredictions)

dfWithPredictions.groupBy("final_status","predictions").count.show()

println("F1 Score est " + score)

+------------+-----------+-----+
|final_status|predictions|count|
+------------+-----------+-----+
|           1|        0.0| 1049|
|           0|        0.0| 4493|
|           1|        1.0| 2361|
|           0|        1.0| 2899|
+------------+-----------+-----+

F1 Score est 0.647367360180229


score: Double = 0.647367360180229


In [81]:
// Saving model

validationModel.save("/home/p5hngk/Downloads/GitHub/INF_729---Introduction_au_framework_Hadoop/cours-spark-telecom-master/model/LogisticRegression")

***Remarque :*** On peut également évaluer la précision avec le modèle suivant

In [79]:
// Evaluer la precision (accuracy)
    val evaluator_acc = new MulticlassClassificationEvaluator()
      .setLabelCol("final_status")
      .setPredictionCol("predictions")
      .setMetricName("accuracy")

    // obtention de la mesure de performance
    val accuracy = evaluator_acc.evaluate(dfWithPredictions)
    println("Precision obtenue : " + accuracy)

Precision obtenue : 0.6345121273838178


evaluator_acc: org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator = mcEval_3717f023f7bb
accuracy: Double = 0.6345121273838178


-----------------------------------------
----------------------------------------
------------------------------------

In [40]:
// """"""""""""""""""""""""" A SUPPRIMER """""""""""""""""""""""""""""""""
validationModel.getEstimatorParamMaps

res15: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	cntVec_e5f43fbe87b3-minDF: 55.0,
	logreg_e2b134433876-regParam: 1.0E-7
}, {
	cntVec_e5f43fbe87b3-minDF: 75.0,
	logreg_e2b134433876-regParam: 1.0E-7
}, {
	cntVec_e5f43fbe87b3-minDF: 95.0,
	logreg_e2b134433876-regParam: 1.0E-7
}, {
	cntVec_e5f43fbe87b3-minDF: 55.0,
	logreg_e2b134433876-regParam: 1.0E-5
}, {
	cntVec_e5f43fbe87b3-minDF: 75.0,
	logreg_e2b134433876-regParam: 1.0E-5
}, {
	cntVec_e5f43fbe87b3-minDF: 95.0,
	logreg_e2b134433876-regParam: 1.0E-5
}, {
	cntVec_e5f43fbe87b3-minDF: 55.0,
	logreg_e2b134433876-regParam: 0.001
}, {
	cntVec_e5f43fbe87b3-minDF: 75.0,
	logreg_e2b134433876-regParam: 0.001
}, {
	cntVec_e5f43fbe87b3-minDF: 95.0,
	logreg_e2b134433876-regParam: 0.001
}, {
	cntVec_e5f43fbe87b3-minDF: 55.0,
	logreg_e2b134...

In [44]:
validationModel

res19: String =
estimator: estimator for selection (current: pipeline_c014b6d0b36d)
estimatorParamMaps: param maps for the estimator (current: [Lorg.apache.spark.ml.param.ParamMap;@1c610ab0)
evaluator: evaluator used to select hyper-parameters that maximize the validated metric (current: mcEval_d6f6bd8301b5)
seed: random seed (default: -1772833110)
trainRatio: ratio between training set and validation set (>= 0 && <= 1) (default: 0.75, current: 0.7)


In [78]:
validationModel.bestModel.asInstanceOf[PipelineModel].stages(10).asInstanceOf[GBTClassificationModel].extractParamMap()

<console>: 43: error: not found: type PipelineModel

In [71]:
validationModel.bestModel.toString

res43: String = pipeline_c014b6d0b36d
